1.使用Titanic数据集，通过特征筛选法一步步提升决策树的预测性能

In [7]:
# 导入数据
import pandas as pd
titanic=pd.read_csv('/Users/scarlett/repository/projects/titanic/titanic.csv')
print titanic.shape
print titanic.info()

# 分离数据特征与预测目标
y=titanic['survived']
X=titanic.drop(['row.names','name','survived'],axis=1)

# 填充缺失数据
X['age'].fillna(X['age'].mean(),inplace=True)
X.fillna('UNKNOWN',inplace=True)

# 分割数据
from sklearn.cross_validation import train_test_split
X_train,y_train,X_test,y_test=train_test_split(X,y,test_size=0.25,random_state=33)

(1313, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB
None


In [ ]:
# 类别型特征向量化
from sklearn.feature_extraction import DictVectorizer
vec=DictVectorizer()
X_train=vec.fit_transform(X_train.to_dict(orient='record'))
X_test=vec.transform(X_test.to_dict(orient='record'))
print len(vec.feature_names_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
dt.score(X_test,y_test)

In [ ]:
# 导入特征筛选器
from sklearn import feature_selection
fs=feature_selection.SelectPercentile(feature_selection.chi2,percentile=20)
X_train_fs=fs.fit_transform(X_train,y_train)
dt.fit(X_train_fs,y_train)
X_test_fs=fs.transform(X_test)
dt.score(X_test_fs,y_test)

In [ ]:
# 通过交叉验证法，按照固定间隔的百分比筛选特征，并作图展示性能岁特征筛选比例的变化
from sklearn.cross_validation import cross_val_score
import numpy as np
percentile=range(1,100,2)
results=[]

for i in percentile:
    fs=feature_selection.SelectPercentile(feature_selection.chi2,percentile=i)
    X_train_fs=fs.fit_transform(X_train,y_train)
    scores=cross_val_score(dt,X_train_fs,y_train,cv=5)
    results=np.append(results,scores.mean())
    print results
    
# 找到提现最佳性能的特征筛选的百分比
opt=np.where(results==results.max())[0]
print 'Optimal number of features %d'%percentiles[opt]